In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import KShape
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from seaborn import heatmap
import pandas as pd
import sklearn
import os
from scipy import signal
import joblib
from sklearn.feature_selection import SelectKBest,f_regression,chi2
from collections import Counter

C:\Users\IDSL\anaconda3\envs\Eisai\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


每週資料加總

In [2]:
def week_data_sum(data):
#     刪除第53週
    data = data.drop(data[data['週'] == 53].index)
    part_compute = pd.DataFrame()
    total_sum = 0
    for num in range(1, 53,1):
        temp = data[data['週'] == num]
        last_temp = temp.tail(1)
        if len(temp) == 0:
            
            last_temp =  data.tail(1)
            last_temp['數量'] = 0
            last_temp['週'] = num
            print("沒有:",last_temp)
            part_compute = part_compute.append(last_temp)
        else:
            temp = temp.drop(temp[temp['數量'] < 0].index)
            last_temp['數量'] =  temp['數量'].sum()
            part_compute = part_compute.append(last_temp)
        total_sum = total_sum +part_compute['數量'].values[0]
    print(part_compute)
    print(total_sum)
   
    
    return part_compute

重新計算工作日

In [3]:
def work_day(data,year):
#     先刪除工作日和休假日
    data = data.drop(['工作天數', '休假天數'], axis=1)
    
    check_date = pd.read_csv(f"../使用量計算/週期資料.csv",encoding='utf-8')
    answer =  pd.DataFrame()
    date =  pd.DataFrame()
    date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
    date['週'] = date['日期'].dt.isocalendar().week
    date['年'] = date['日期'].dt.isocalendar().year
    date = date.drop(date[date['週'] == 53].index).reset_index()
    del_data =  date[date['週'] == 52].tail(1).index
    date = date.drop(date[del_data.values[0]+1:].index).reset_index()
    date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
    check_date["date"] = pd.to_datetime(check_date["date"] ,format='%Y/%m/%d')
    # 如果是工作日就存成true，否則存成false
    date['休假日']=date['日期'].map(lambda x:(check_date['date']==x).any())
    print(date)

    for num in range(1, 53,1):
        temp = date[date['週'] == num]
#         計算該周工作日
        holiday = len(temp[temp['休假日'] == True])
        work_day = len(temp[temp['休假日'] == False])
        date.loc[date['週']==num,'工作天數'] = work_day
        date.loc[date['週']==num,'休假天數'] = holiday
        if year == 2021 and num == 52:   
            date.loc[date['週']==num,'休假天數'] = 2

#     同一週期工作日、休假日都一樣所以只保留一筆
    date.drop_duplicates(subset='週', keep='last', inplace=True)
    date = date.drop(['日期', '休假日','level_0','index'], axis=1)
  
#     answer = pd.merge(data, date, on='週',how='outer')

    answer = pd.merge(data, date, on='週')
#     answer = answer.sort_values(['帳務日期'], ascending=True).reset_index(drop=True)

#     刪除日期和工作日
    answer.rename(columns={'年_x': '年'}, inplace=True)
    
    return answer

正規化

In [4]:
#正規化
def data_normalize(total_data, need_normalize):
    data = total_data.copy()
    train_data = data[(data['年']== 2019)|(data['年']== 2020)|(data['年']== 2021)]     
#     train_data = data[data['年']== 2019]     
    #使用最大最小值進行標準化
    for i in range(len(need_normalize)):
        column = need_normalize[i]
        molecular = data[column]-train_data[column].min()
        denominator = train_data[column].max()-train_data[column].min()
        data[column] = (molecular/denominator)
    return data

In [5]:

# A00120033合併檔
# A00120067合併檔
# A00120110合併檔
# A04200004合併檔

x_train = []

part_no_total =['B03106859','C13930399','A00120213','A02320340','A02322259','A04411285','A04800036','B00206057','B03110120','G81500272',
                'K80004044','A00120033合併檔','A00120067合併檔','A00120110合併檔','A04200004合併檔']
# part_no_total = ["A00120033","A00120067","A00120110","A00120213","A02120180","A02320340","A02322259","A04200004","A04411285","A04800036","B00206057",
#                 "B00304056","B03106859","B03110120","C13930399","G81500272","K80004044"]






for part_no in part_no_total:
    data=pd.read_csv(f"../使用量計算/有加工作日/五病房/{part_no}.csv")
    data = data[data.columns.drop(list(data.filter(regex='Unnamed')))]
    # data['年'] =  pd.to_datetime(data['帳務日期']).dt.year

    data_2019 = data[data['年']==2019]
    data_2020 = data[data['年']==2020]
    data_2021 = data[data['年']==2021]
    data_2022 = data[data['年']==2022]


    # 把每一週的使用量加總
    data_2019 = week_data_sum(data_2019)
    data_2019 = work_day(data_2019,2019)
    data_2020 = week_data_sum(data_2020)
    data_2020 = work_day(data_2020,2020)
    data_2021 = week_data_sum(data_2021)
    data_2021 = work_day(data_2021,2021)
    data_2022 = week_data_sum(data_2022)
    data_2022 = work_day(data_2022,2022)
    
    total_data = pd.concat([data_2019, data_2020])
    total_data = pd.concat([total_data, data_2021])
    total_data = pd.concat([total_data, data_2022])
    total_data = total_data.reset_index()
    
    total_data = total_data[(total_data['年']== 2019)|(total_data['年']== 2020)|(total_data['年']== 2021)]     
    total_data = data_normalize(total_data, ["數量"])
    
    x_train.append(total_data["數量"].values)
    print("測試",total_data.head())


C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users

        庫房         料號    品名     數量        帳務日期   週    工作日     年  工作天數  休假天數
67    DLN5  B03106859  人工薄膜   74.0  2019-12-31   1   True  2019   3.0   3.0
143   DLN5  B03106859  人工薄膜   95.0  2019-01-13   2  False  2019   5.0   2.0
202   DLN5  B03106859  人工薄膜   67.0  2019-01-20   3  False  2019   5.0   2.0
261   DLN5  B03106859  人工薄膜   67.0  2019-01-27   4  False  2019   5.0   2.0
295   DLN5  B03106859  人工薄膜   37.0  2019-02-03   5   True  2019   5.0   2.0
342   DLN5  B03106859  人工薄膜   53.0  2019-02-10   6  False  2019   0.0   7.0
421   DLN5  B03106859  人工薄膜   85.0  2019-02-17   7  False  2019   5.0   2.0
487   DLN5  B03106859  人工薄膜   80.0  2019-02-24   8  False  2019   5.0   2.0
542   DLN5  B03106859  人工薄膜   69.0  2019-03-03   9  False  2019   3.0   4.0
622   DLN5  B03106859  人工薄膜   94.0  2019-03-10  10  False  2019   5.0   2.0
686   DLN5  B03106859  人工薄膜   78.0  2019-03-17  11  False  2019   5.0   2.0
770   DLN5  B03106859  人工薄膜  109.0  2019-03-24  12  False  2019   5.0   2.0
828   DLN5  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users

        庫房         料號    品名     數量        帳務日期   週    工作日     年  工作天數  休假天數
3367  DLN5  B03106859  人工薄膜   36.0  2020-01-05   1  False  2020   2.0   3.0
3407  DLN5  B03106859  人工薄膜   46.0  2020-01-12   2  False  2020   5.0   2.0
3460  DLN5  B03106859  人工薄膜   60.0  2020-01-19   3   True  2020   5.0   2.0
3471  DLN5  B03106859  人工薄膜   12.0  2020-01-22   4   True  2020   3.0   4.0
3484  DLN5  B03106859  人工薄膜   13.0  2020-02-02   5  False  2020   2.0   5.0
3560  DLN5  B03106859  人工薄膜   87.0  2020-02-09   6  False  2020   5.0   2.0
3628  DLN5  B03106859  人工薄膜   74.0  2020-02-16   7  False  2020   5.0   2.0
3701  DLN5  B03106859  人工薄膜   86.0  2020-02-23   8  False  2020   5.0   2.0
3754  DLN5  B03106859  人工薄膜   56.0  2020-03-01   9  False  2020   4.0   3.0
3816  DLN5  B03106859  人工薄膜   69.0  2020-03-08  10  False  2020   5.0   2.0
3889  DLN5  B03106859  人工薄膜   85.0  2020-03-15  11  False  2020   5.0   2.0
3959  DLN5  B03106859  人工薄膜   79.0  2020-03-22  12  False  2020   5.0   2.0
4049  DLN5  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users

...

        庫房         料號                    品名    數量        帳務日期   週    工作日  \
2365  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  20.0  2020-01-05   1  False   
2404  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  41.0  2020-01-12   2  False   
2447  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  45.0  2020-01-19   3   True   
2455  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag   8.0  2020-01-22   4   True   
2468  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  13.0  2020-01-31   5  False   
2521  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  56.0  2020-02-09   6  False   
2574  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  54.0  2020-02-16   7  False   
2623  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  49.0  2020-02-23   8  False   
2670  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  47.0  2020-03-01   9  False   
2714  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  44.0  2020-03-08  10  False   
2768  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  54.0  2020-03-15  11  False   
2814  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  47.0  2020-03-22  12  False   
2869  DLN5  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users

        庫房         料號                    品名    數量        帳務日期   週    工作日  \
5121  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  64.0  2021-01-10   1  False   
5192  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  75.0  2021-01-17   2  False   
5250  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  67.0  2021-01-24   3  False   
5310  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  67.0  2021-01-31   4  False   
5364  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  58.0  2021-02-07   5   True   
5373  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag   9.0  2021-02-09   6   True   
5406  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  33.0  2021-02-20   7   True   
5482  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  81.0  2021-02-28   8  False   
5530  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  56.0  2021-03-07   9  False   
5583  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  53.0  2021-03-14  10  False   
5638  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  57.0  2021-03-21  11  False   
5684  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  48.0  2021-03-28  12  False   
5730  DLN5  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  part_compute = part_compute.append(last_temp)
C:\Users

     level_0  index         日期   週     年    休假日
0          0      0 2022-01-01  52  2021   True
1          1      1 2022-01-02  52  2021   True
2          2      2 2022-01-03   1  2022  False
3          3      3 2022-01-04   1  2022  False
4          4      4 2022-01-05   1  2022  False
..       ...    ...        ...  ..   ...    ...
360      360    360 2022-12-27  52  2022  False
361      361    361 2022-12-28  52  2022  False
362      362    362 2022-12-29  52  2022  False
363      363    363 2022-12-30  52  2022  False
364      364    364 2022-12-31  52  2022   True

[365 rows x 6 columns]
測試    index    庫房         料號                    品名        數量        帳務日期  週  \
0      0  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  0.575342  2019-12-31  1   
1      1  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  0.602740  2019-01-13  2   
2      2  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  0.534247  2019-01-20  3   
3      3  DLN5  A04200004  精密輸液套附Y加藥座120c.c硬bag  0.547945  2019-01-27  4   
4      4  DLN5  A

取值

In [6]:
def take_part(pred,part_no_total):
    answer = []
    for num in pred:
#         print(num)
        answer.append(part_no_total[num])
    
    return answer 

In [7]:
# Euclidean k-means
#分幾類
clusters_number = 5
ks = KShape(n_clusters=clusters_number, verbose=False, random_state=1)
y_pred = ks.fit_predict(x_train)
# 印出第n類
for i in range(0,clusters_number):
    y_pred_2 = np.where(y_pred == i)

    series = take_part(y_pred_2[0],part_no_total)
    print(series)



['A04800036']
['A00120213']
['A02322259']
['B03106859', 'C13930399', 'A02320340', 'B00206057', 'K80004044', 'A00120033合併檔', 'A00120067合併檔', 'A00120110合併檔', 'A04200004合併檔']
['A04411285', 'B03110120', 'G81500272']


In [8]:
for i in range(0,clusters_number):
    print(i)

0
1
2
3
4


In [9]:
# 找出組合並且把組合儲存陣列
count_class = Counter(y_pred)
count_class = list(count_class.keys())
count_class = count_class[count_class == 2]
count_class

3

In [10]:
count_class

3

In [11]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
line_color = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

My_list = [*range(1, 53, 1)]
part_no_total =['A04411285', 'B00304056', 'G81500272']

total_picture = []

for part_no in part_no_total:
    data=pd.read_csv(f"../使用量計算/有加工作日/五病房/{part_no}.csv")
    data = data[data.columns.drop(list(data.filter(regex='Unnamed')))]
    # data['年'] =  pd.to_datetime(data['帳務日期']).dt.year

    data_2019 = data[data['年']==2019]
    data_2020 = data[data['年']==2020]
    data_2021 = data[data['年']==2021]
    # 把每一週的使用量加總
    data_2019 = week_data_sum(data_2019)
    data_2020 = week_data_sum(data_2020)
    data_2021 = week_data_sum(data_2021)
    total_data = pd.concat([data_2019, data_2020])
    total_data = pd.concat([total_data, data_2021])
    total_data = total_data.reset_index()

    data_2019 =  total_data[total_data['年'] == 2019]
    total_picture.append(data_2019['數量'].values)
    



fig_line = go.Figure()
fig_line.add_trace(go.Scatter(y = total_picture[0] , x=My_list,
                    mode='lines',
                    name=f'A04411285',
                    line={'dash': 'dash'},
                    line_color= '#1f77b4'))

fig_line.add_trace(go.Scatter(y = total_picture[1], x=My_list,
                    mode='lines',
                    name='B00304056',
                    line_color= '#ff7f0e'))

fig_line.add_trace(go.Scatter(y =total_picture[2], x=My_list,
                    mode='lines',
                    name='G81500272',
                    line_color= '#ff0e16'))
# fig_line.add_trace(go.Scatter(y = total_picture[3], x=My_list,
#                     mode='lines',
#                     name='A00120033',
#                     line_color= '#7f7f7f'))



fig_line.update_layout(
    yaxis_title='用量',
    xaxis_title='日期',
    title='使用量',
    font=dict(
        size=18,
    ),
#     yaxis2=dict(anchor='x', overlaying='y', side='right')
    height=450, 
    width=1500,

)

fig_line.update_xaxes(nticks=52)


#     fig_line.write_html(f'{folder_path}/img/{methods}_{i}.html')

fig_line.show()

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

        庫房         料號      品名     數量        帳務日期   週    工作日     年  工作天數  休假天數
66    DLN5  A04411285  全拋式採血器  135.0  2019-12-31   1   True  2019   3.0   3.0
139   DLN5  A04411285  全拋式採血器  170.0  2019-01-13   2  False  2019   5.0   2.0
214   DLN5  A04411285  全拋式採血器  157.0  2019-01-20   3  False  2019   5.0   2.0
270   DLN5  A04411285  全拋式採血器  121.0  2019-01-27   4  False  2019   5.0   2.0
315   DLN5  A04411285  全拋式採血器  109.0  2019-02-03   5   True  2019   5.0   2.0
386   DLN5  A04411285  全拋式採血器  235.0  2019-02-10   6  False  2019   0.0   7.0
477   DLN5  A04411285  全拋式採血器  228.0  2019-02-17   7  False  2019   5.0   2.0
573   DLN5  A04411285  全拋式採血器  217.0  2019-02-24   8  False  2019   5.0   2.0
675   DLN5  A04411285  全拋式採血器  246.0  2019-03-03   9  False  2019   3.0   4.0
753   DLN5  A04411285  全拋式採血器  198.0  2019-03-10  10  False  2019   5.0   2.0
802   DLN5  A04411285  全拋式採血器   98.0  2019-03-17  11  False  2019   5.0   2.0
866   DLN5  A04411285  全拋式採血器  156.0  2019-03-24  12  False  201

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\IDSL\AppData\Local\Temp\ipykernel_18700\2613971702.py:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

FileNotFoundError: [Errno 2] No such file or directory: '../使用量計算/有加工作日/五病房/B00304056.csv'